# Audio Quality Analysis - Batch Processing

**Goal:** Calculate audio quality metrics for ALL VHP files to identify audio issues and guide preprocessing decisions.

**Output:** DataFrame with per-file metrics + detected issues → saved as `audio_quality_analysis.parquet`

**Metrics:**
- SNR (signal-to-noise ratio)
- Spectral roll-off (bandwidth indicator)
- Spectral centroid (brightness)
- Spectral flatness (tonal vs noisy)
- ZCR (zero crossing rate - noisiness)
- Loudness (LUFS)

**Issue Detection:**
- `bandwidth_limited`: Roll-off < 1000 Hz (missing high frequencies)
- `high_noise`: ZCR > 0.05 or SNR < 15 dB
- `low_frequency_rumble`: High energy below 80 Hz
- `low_loudness`: LUFS < -30 dB

In [ ]:
import sys
sys.path.append("../scripts")

import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pyloudnorm as pyln
from tqdm import tqdm
import io
import tempfile
from pydub import AudioSegment

# Azure blob utilities
from scripts.cloud.azure_utils import list_blobs, download_blob_to_memory

%matplotlib inline

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='../credentials/creds.env')

In [ ]:
# Verify Azure authentication environment variables
import os

print("Azure Authentication Configuration:")
print("=" * 60)
print(f"AZURE_STORAGE_ACCOUNT: {os.getenv('AZURE_STORAGE_ACCOUNT')}")
print(f"AZURE_STORAGE_CONTAINER: {os.getenv('AZURE_STORAGE_CONTAINER')}")
print(f"AZURE_AUTH: {os.getenv('AZURE_AUTH')}")
print(f"Connection string set: {'Yes' if os.getenv('AZURE_STORAGE_CONNECTION_STRING') else 'No'}")
print("=" * 60)

# Check if all required vars are set
required_vars = ['AZURE_STORAGE_ACCOUNT', 'AZURE_STORAGE_CONTAINER', 'AZURE_AUTH']
missing = [v for v in required_vars if not os.getenv(v)]

if missing:
    print(f"\n⚠️  WARNING: Missing environment variables: {', '.join(missing)}")
    print("Please set these before running the notebook:")
    print("  export AZURE_STORAGE_ACCOUNT=stgamiadata26828")
    print("  export AZURE_STORAGE_CONTAINER=audio-raw")
    print("  export AZURE_AUTH=connection_string")
    print("  export AZURE_STORAGE_CONNECTION_STRING='...'")
else:
    print("\n✓ All required environment variables are set!")

## 1. Audio Quality Metric Functions

Functions to calculate audio quality metrics from waveform.

In [ ]:
def snr_cal(wv, sr) -> np.float32:
    """Calculate Signal-to-Noise Ratio (dB)"""
    rms_full = librosa.feature.rms(y=wv)
    noise_rms = np.mean(rms_full[:, :int(0.5*sr/512)])
    signal_rms = np.mean(rms_full)
    snr_db = 20 * np.log10(signal_rms / noise_rms)
    return snr_db

def spectral_rolloff_cal(wv, sr) -> np.float64:
    """Calculate spectral roll-off (85th percentile frequency)"""
    rolloff = librosa.feature.spectral_rolloff(y=wv, sr=sr, roll_percent=0.85)
    return np.median(rolloff)

def spectral_flatness_cal(wv) -> np.float64:
    """Calculate spectral flatness (0=tonal, 1=noisy)"""
    flatness = librosa.feature.spectral_flatness(y=wv)
    return np.mean(flatness)

def spectral_centroid_cal(wv, sr) -> np.float64:
    """Calculate spectral centroid (brightness)"""
    centroid = librosa.feature.spectral_centroid(y=wv, sr=sr)
    return np.median(centroid)

def zcr_cal(wv):
    """Calculate zero crossing rate (mean and variance)"""
    zcr = librosa.feature.zero_crossing_rate(wv)
    zcr_mean = np.mean(zcr)
    zcr_var = np.var(zcr)
    return zcr_mean, zcr_var

def loudness_cal(wv, sr) -> np.float64:
    """Calculate loudness (LUFS - ITU-R BS.1770-4)"""
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(wv)
    return loudness

def low_frequency_energy(wv, sr, cutoff_hz=80) -> np.float64:
    """Calculate energy below cutoff frequency (for rumble detection)"""
    # Compute STFT
    D = np.abs(librosa.stft(wv))
    freqs = librosa.fft_frequencies(sr=sr)
    
    # Energy below cutoff
    low_freq_mask = freqs < cutoff_hz
    low_energy = np.sum(D[low_freq_mask, :])
    total_energy = np.sum(D)
    
    # Return ratio
    return low_energy / total_energy if total_energy > 0 else 0.0

## 2. Batch Audio Analysis Function

Process one audio file and return metrics.

In [ ]:
def analyze_audio_file(audio_bytes, audio_id: str) -> dict:
    """
    Analyze audio file and return quality metrics.
    
    Handles both audio (MP3, WAV) and video (MP4) files using pydub.
    
    Args:
        audio_bytes: Audio/video file as bytes
        audio_id: Identifier for the audio file
    
    Returns:
        Dictionary with audio_id and quality metrics
    """
    try:
        # Use pydub to load audio (handles MP3, MP4, M4A, WAV, etc.)
        # This extracts audio stream from video files like MP4
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes))
        
        # Convert to mono and 16kHz (consistent with pipeline)
        audio_segment = audio_segment.set_channels(1)
        audio_segment = audio_segment.set_frame_rate(16000)
        
        # Convert to numpy array
        samples = np.array(audio_segment.get_array_of_samples())
        
        # Normalize to float32 [-1, 1]
        if audio_segment.sample_width == 2:  # 16-bit
            wv = samples.astype(np.float32) / 32768.0
        elif audio_segment.sample_width == 4:  # 32-bit
            wv = samples.astype(np.float32) / 2147483648.0
        else:
            wv = samples.astype(np.float32)
        
        sr = 16000  # Sample rate after conversion
        
        # Calculate metrics
        snr = float(snr_cal(wv, sr))
        rolloff = float(spectral_rolloff_cal(wv, sr))
        flatness = float(spectral_flatness_cal(wv))
        centroid = float(spectral_centroid_cal(wv, sr))
        zcr_mean, zcr_var = zcr_cal(wv)
        zcr_mean = float(zcr_mean)
        zcr_var = float(zcr_var)
        loudness = float(loudness_cal(wv, sr))
        low_freq_energy_ratio = float(low_frequency_energy(wv, sr, cutoff_hz=80))
        
        # Duration
        duration_sec = len(wv) / sr
        
        return {
            'audio_id': audio_id,
            'sample_rate': sr,
            'duration_sec': duration_sec,
            'snr_db': snr,
            'spectral_rolloff_hz': rolloff,
            'spectral_flatness': flatness,
            'spectral_centroid_hz': centroid,
            'zcr_mean': zcr_mean,
            'zcr_var': zcr_var,
            'loudness_lufs': loudness,
            'low_freq_energy_ratio': low_freq_energy_ratio,
            'status': 'success'
        }
    
    except Exception as e:
        return {
            'audio_id': audio_id,
            'status': 'error',
            'error_message': str(e)
        }

## 3. Issue Detection Logic

Detect audio issues based on metric thresholds.

In [ ]:
def detect_audio_issues(row: pd.Series) -> list:
    """
    Detect audio issues based on quality metrics.
    
    Args:
        row: DataFrame row with audio metrics
    
    Returns:
        List of detected issues
    """
    issues = []
    
    if row['status'] != 'success':
        return issues
    
    # 1. Bandwidth-limited (lacking high frequencies)
    if row['spectral_rolloff_hz'] < 1000:
        issues.append('bandwidth_limited_severe')
    elif row['spectral_rolloff_hz'] < 4000:
        issues.append('bandwidth_limited_moderate')
    
    # 2. High noise
    if row['zcr_mean'] > 0.05:
        issues.append('high_noise_zcr')
    if row['snr_db'] < 15:
        issues.append('high_noise_snr')
    
    # 3. Low-frequency rumble
    if row['low_freq_energy_ratio'] > 0.15:
        issues.append('low_frequency_rumble')
    
    # 4. Low loudness
    if row['loudness_lufs'] < -30:
        issues.append('low_loudness')
    
    # 5. Very flat spectrum
    if row['spectral_flatness'] > 0.8:
        issues.append('very_flat_spectrum')
    
    return issues


def recommend_preprocessing(issues: list) -> list:
    """
    Recommend preprocessing methods based on detected issues.
    
    Args:
        issues: List of detected issues
    
    Returns:
        List of recommended preprocessing methods
    """
    recommendations = []
    
    # Always normalize loudness
    recommendations.append('loudness_normalization')
    
    # Bandwidth-limited → EQ boost high frequencies
    if 'bandwidth_limited_severe' in issues or 'bandwidth_limited_moderate' in issues:
        recommendations.append('eq_high_freq_boost')
    
    # High noise → Noise reduction
    if 'high_noise_zcr' in issues or 'high_noise_snr' in issues:
        recommendations.append('noise_reduction')
    
    # Low-frequency rumble → High-pass filter
    if 'low_frequency_rumble' in issues:
        recommendations.append('highpass_filter')
    
    return recommendations

## 4. Batch Process All VHP Audio Files

Download from Azure Blob and analyze.

In [ ]:
# List all audio/video blobs from Azure
blob_prefix = "loc_vhp/"  # Updated to match your actual blob structure

print(f"Listing blobs with prefix: {blob_prefix}")
audio_blobs = list_blobs(blob_prefix)

# Filter for audio AND video files (pipeline handles both MP3 and MP4)
media_extensions = ('.mp3', '.mp4', '.wav', '.m4a', '.flac', '.ogg')
audio_blobs = [b for b in audio_blobs if b.lower().endswith(media_extensions)]

print(f"Found {len(audio_blobs)} media files (audio + video)")
print(f"\nFirst 5 files:")
for blob in audio_blobs[:5]:
    print(f"  - {blob}")

In [ ]:
# TESTING: Limit to small sample for quick testing
# Comment out this line once you've verified everything works
SAMPLE_SIZE = 3  # Process only first 3 files for testing
audio_blobs = audio_blobs[:SAMPLE_SIZE]
print(f"\n🧪 TEST MODE: Processing only first {SAMPLE_SIZE} files")
print("Once verified, comment out the SAMPLE_SIZE lines above to process all files\n")

In [ ]:
# Process all audio files (streaming - one at a time)
# Only stores metrics, not audio data
results = []

print(f"\nProcessing {len(audio_blobs)} audio files (streaming)...\n")
print("Memory usage: Only stores metrics (~2-3 KB per file), not audio\n")

for blob_path in tqdm(audio_blobs):
    audio_id = Path(blob_path).stem
    
    try:
        # Download ONE file, analyze, discard audio, keep metrics
        audio_bytes = download_blob_to_memory(blob_path)
        result = analyze_audio_file(audio_bytes, audio_id)
        results.append(result)  # Only stores ~12 numbers per file
        
        # audio_bytes is garbage collected after this iteration
        
    except Exception as e:
        print(f"Error processing {blob_path}: {e}")
        results.append({
            'audio_id': audio_id,
            'status': 'error',
            'error_message': str(e)
        })

print(f"\nProcessed {len(results)} files")
print(f"Memory used for results: ~{len(results) * 0.3:.1f} KB (metrics only)")

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(results)

print(f"Total files: {len(df)}")
print(f"Successful: {(df['status'] == 'success').sum()}")
print(f"Errors: {(df['status'] == 'error').sum()}")

df.head()

## 5. Detect Issues and Recommend Preprocessing

In [ ]:
# Detect issues
df['issues'] = df.apply(detect_audio_issues, axis=1)
df['recommended_preprocessing'] = df['issues'].apply(recommend_preprocessing)

df_with_issues = df[df['issues'].str.len() > 0]

print(f"Files with issues: {len(df_with_issues)} / {len(df)}")

from collections import Counter
all_issues = [issue for issues in df['issues'] for issue in issues]
issue_counts = Counter(all_issues)

print("\nIssue breakdown:")
for issue, count in issue_counts.most_common():
    print(f"  {issue}: {count} files ({count/len(df)*100:.1f}%)")

In [ ]:
# Sample with issues
df_with_issues[['audio_id', 'spectral_rolloff_hz', 'snr_db', 'zcr_mean', 
                 'loudness_lufs', 'issues', 'recommended_preprocessing']].head(10)

## 6. Visualize Metrics Distribution

In [ ]:
df_success = df[df['status'] == 'success']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].hist(df_success['snr_db'], bins=30, edgecolor='black')
axes[0, 0].set_xlabel('SNR (dB)')
axes[0, 0].set_title('Signal-to-Noise Ratio')
axes[0, 0].axvline(x=15, color='r', linestyle='--', label='Threshold')
axes[0, 0].legend()

axes[0, 1].hist(df_success['spectral_rolloff_hz'], bins=30, edgecolor='black')
axes[0, 1].set_xlabel('Spectral Roll-off (Hz)')
axes[0, 1].set_title('Bandwidth Indicator')
axes[0, 1].axvline(x=1000, color='r', linestyle='--', label='Severe')
axes[0, 1].axvline(x=4000, color='orange', linestyle='--', label='Moderate')
axes[0, 1].legend()

axes[0, 2].hist(df_success['zcr_mean'], bins=30, edgecolor='black')
axes[0, 2].set_xlabel('ZCR Mean')
axes[0, 2].set_title('Zero Crossing Rate')
axes[0, 2].axvline(x=0.05, color='r', linestyle='--', label='Threshold')
axes[0, 2].legend()

axes[1, 0].hist(df_success['loudness_lufs'], bins=30, edgecolor='black')
axes[1, 0].set_xlabel('Loudness (LUFS)')
axes[1, 0].set_title('Loudness')
axes[1, 0].axvline(x=-30, color='r', linestyle='--', label='Low')
axes[1, 0].axvline(x=-23, color='g', linestyle='--', label='Target')
axes[1, 0].legend()

axes[1, 1].hist(df_success['spectral_flatness'], bins=30, edgecolor='black')
axes[1, 1].set_xlabel('Spectral Flatness')
axes[1, 1].set_title('Tonal vs Noisy')

axes[1, 2].hist(df_success['low_freq_energy_ratio'], bins=30, edgecolor='black')
axes[1, 2].set_xlabel('Low Freq Energy')
axes[1, 2].set_title('Rumble Indicator')
axes[1, 2].axvline(x=0.15, color='r', linestyle='--', label='Threshold')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 7. Summary Statistics

In [ ]:
print("Overall Statistics:")
print(df_success[['snr_db', 'spectral_rolloff_hz', 'spectral_flatness', 
                   'zcr_mean', 'loudness_lufs']].describe())

In [ ]:
print("\nPreprocessing Recommendations:")
all_recs = [rec for recs in df['recommended_preprocessing'] for rec in recs]
rec_counts = Counter(all_recs)

for rec, count in rec_counts.most_common():
    print(f"{rec:30s}: {count:4d} files ({count/len(df)*100:5.1f}%)")

## 8. Save Results

In [ ]:
output_path = Path("../data/audio_quality_analysis.parquet")
df.to_parquet(output_path, index=False)

print(f"Saved: {output_path}")
print(f"Rows: {len(df)}")
print(f"Columns: {list(df.columns)}")